# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weatherpy_data = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Drop unnamed column (unnecessary)

weatherpy_df = weatherpy_data.drop(columns=["Unnamed: 0"])

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weatherpy_df[['Latitude','Longtitude']]
humidity = weatherpy_df['Humidity'].astype(float)

fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(locations, weights = humidity,
                             dissipating = False, max_intensity = 10,
                             point_radius = 1)

# Add heatmap layer to the map
fig.add_layer(heatmap)
# display figure
fig

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
ideal_cities = pd.DataFrame(weatherpy_data, columns = ["City", "Country", "Latitude", "Longtitude", "Max Temp", "Humidity", "Cloudiness"])

max_temp = (ideal_cities["Max Temp"] <= 90) & (ideal_cities["Max Temp"] >= 50)
humidity = (ideal_cities["Humidity"] < 50)
cloudiness = (ideal_cities["Cloudiness"] <= 50)
lat_ideal = (ideal_cities["Latitude"])
long_ideal= (ideal_cities["Longtitude"])

ideal_cities_df= ideal_cities[max_temp & humidity & cloudiness]
ideal_cities_df.head()

,City,Country,Latitude,Longtitude,Max Temp,Humidity,Cloudiness
9,San Luis,AR,-33.2950,-66.3356,74.17,39,35
92,Mandsaur,IN,24.0667,75.0667,59.29,35,0
126,Nishihara,JP,26.1842,127.7558,71.60,37,20
130,Dwārka,IN,22.2394,68.9678,69.31,48,0
136,Lüderitz,NaN,-26.6481,15.1594,68.05,47,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
hotel_df = pd.DataFrame(ideal_cities_df, columns =["City", "Latitude", "Longtitude"])

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Latitude,Longtitude,Hotel Name
9,San Luis,-33.2950,-66.3356,
92,Mandsaur,24.0667,75.0667,
126,Nishihara,26.1842,127.7558,
130,Dwārka,22.2394,68.9678,
136,Lüderitz,-26.6481,15.1594,


In [66]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longtitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 9: San Luis.
Closest hotel is Huarpes Hotel.
------------
Retrieving Results for Index 92: Mandsaur.
Closest hotel is Hotel Galaxy Inn.
------------
Retrieving Results for Index 126: Nishihara.
Closest hotel is Minshuku Agaihama.
------------
Retrieving Results for Index 130: Dwārka.
Closest hotel is Hotel Narayan Avezika Comfort.
------------
Retrieving Results for Index 136: Lüderitz.
Closest hotel is LÜDERITZ NEST HOTEL.
------------
Retrieving Results for Index 139: Saint George.
Closest hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 148: Kothāpet.
Closest hotel is Andhavelly Village.
------------
Retrieving Results for Index 154: Ures.
Closest hotel is Hotel HACIENDA San Francisco.
------------
Retrieving Results for Index 155: Āmlāgora.
Closest hotel is Mohul Guest House.
------------
Retrieving Results for Index 186: Marzuq.
Missing field/result... skipping.
------------
Retrieving Results for Index 187: Boulsa.
Close

In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
